In [1]:
import numpy as np
import matplotlib.pyplot as plt
import autokeras as ak
import tensorflow as tf
import os

In [2]:
import logging

logging.basicConfig(level = logging.INFO)
gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   # Restrict TensorFlow to only use the first GPU
try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2000)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#     logging.info(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
except RuntimeError as e:
# Visible devices must be set before GPUs have been initialized
    logging.info(e)

In [3]:
data = np.load("./nsi_data/sample_nsi_regression_1e7_v1.npz")

In [4]:
data_selection = 1 # 0 for all, 1 for lowE(<5GeV), 2 for high(>5GeV)

if data_selection == 0:
    data_all = np.column_stack([data['ve_dune'], data['vu_dune'], data['vebar_dune'], data['vubar_dune']])
elif data_selection == 1:
    data_all = np.column_stack([data['ve_dune'][:,:36], data['vu_dune'][:,:36], data['vebar_dune'][:,:36], data['vubar_dune'][:,:36]])
elif data_selection == 2:
    data_all = np.column_stack([data['ve_dune'][:,36:], data['vu_dune'][:,36:], data['vebar_dune'][:,36:], data['vubar_dune'][:,36:]])

target = np.column_stack([data['theta23'], data['delta']/36])

x_train = data_all[:10000]
y_train = target[:10000]

x_train2 = data_all[10000:9000000]
y_train2 = target[10000:9000000]

x_test = data_all[9000000:]
y_test = target[9000000:]

In [5]:
clf = ak.StructuredDataRegressor(overwrite=True, max_trials=50)
clf.fit(x_train, y_train,
           validation_split = 0.1,
           batch_size=64,
           epochs=20,
           verbose=1,
           shuffle = True
       )

Trial 50 Complete [00h 00m 27s]
val_loss: 2.882599353790283

Best val_loss So Far: 2.882599353790283
Total elapsed time: 00h 22m 22s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Epoch 1/20
157/157 [==============================] - 2s 7ms/step - loss: 673.9763 - mean_squared_error: 673.9763
Epoch 2/20
157/157 [==============================] - 1s 7ms/step - loss: 19.5356 - mean_squared_error: 19.5356
Epoch 3/20
157/157 [==============================] - 1s 7ms/step - loss: 10.2896 - mean_squared_error: 10.2896
Epoch 4/20
157/157 [==============================] - 1s 7ms/step - loss: 7.6564 - mean_squared_error: 7.6564
Epoch 5/20
157/157 [==============================] - 1s 7ms/step - loss: 6.4129 - mean_squared_error: 6.4129
Epoch 6/20
157/157 [==============================] - 1s 8ms/step - loss: 5.4973 - mean_squared_error: 5.4973
Epoch 7/20
157/157 [==============================] - 1s 7ms/step - loss: 5.1635 - mean_squared_error: 5.1635
Epoch 8/20
157/157 [==============================] - 1s 7ms/step - loss: 4.7846 - mean_squared_error: 4.7846
Epoch 9/20
157/157 [==============================] - 1s 7ms/step - loss: 4.3757 - mean_squared_error: 4.3757
Ep


FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.


INFO:tensorflow:Assets written to: ./structured_data_regressor/best_model/assets


INFO:tensorflow:Assets written to: ./structured_data_regressor/best_model/assets


In [6]:
model = clf.export_model()
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 144)]             0         
_________________________________________________________________
multi_category_encoding (Mul (None, 144)               0         
_________________________________________________________________
normalization (Normalization (None, 144)               289       
_________________________________________________________________
dense (Dense)                (None, 512)               74240     
_________________________________________________________________
re_lu (ReLU)                 (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              525312    
_________________________________________________________________
re_lu_1 (ReLU)               (None, 1024)              0     

In [ ]:
model.fit(x_train2, y_train2,
           validation_split = 0.1,
           batch_size=64,
           epochs=20,
           verbose=1,
           shuffle = True
         )

Epoch 1/20
126422/126422 [==============================] - 1029s 8ms/step - loss: 1.4388 - mean_squared_error: 1.4388 - val_loss: 1.4162 - val_mean_squared_error: 1.4162
Epoch 2/20
 98121/126422 [======================>.......] - ETA: 3:52 - loss: 1.4121 - mean_squared_error: 1.4121

In [ ]:
index = 1
while os.path.isfile('./models/0910_2params_{}.h5'.format(index)):
    index += 1
model.save('./models/0910_2params_{}.h5'.format(index))

In [ ]:
scale_steps = np.logspace(-3, 0, 30)
before_train_loss = []
after_train_loss = []

for scale in scale_steps:
    x_train2_gen = np.random.normal(x_train2, np.sqrt(x_train2)*scale)
    x_test_gen = np.random.normal(x_test, np.sqrt(x_test)*scale)

    before_train_loss.append(model.evaluate(x_test_gen, y_test)[0])

    model.fit(x_train2_gen, y_train2,
               validation_split = 0.1,
               batch_size=64,
               epochs=5,
               verbose=1,
               shuffle = True
             )

    after_train_loss.append(model.evaluate(x_test_gen, y_test)[0])

In [ ]:
model_index = index
index = 1
path = './models_furthurTrain/0910_2params_{}_{}.h5'
while os.path.isfile(path.format(model_index, index)):
    index += 1
model.save(path.format(model_index, index))
outfile = {'scale_steps': scale_steps,
           'before_train_loss': before_train_loss,
           'after_train_loss' :after_train_loss}
np.save(file = './models_furthurTrain/0910_2params_{}_{}_result.npy'.format(model_index, index),
        arr = outfile)

In [ ]:
x_test2_gen = np.random.poisson(x_test)

for i in range(10):
    x_train2_gen = np.random.poisson(x_train2)
    
    model.fit(x_train2_gen, y_train2,
              validation_split=0.1,
               batch_size=64,
               epochs=1,
               verbose=1,
               shuffle = True
             )
model.evaluate(x_test2_gen, y_test)

In [ ]:
furthur_index = index
index = 1
path = './models_PoissonTrain/0910_2params_{}_{}_{}.h5'
while os.path.isfile(path.format(model_index, furthur_index, index)):
    index += 1
model.save(path.format(model_index, furthur_index, index))